# Introdução

No problema da mochila temos uma mochila que suporta uma capacidade de peso máxima (K) e vários itens, cada um com seu peso e valor. O objetivo do problema é maximizar o valor obtido dentro da mochila sem estourar sua capacidade máxima.

## Descrição matemática
**1. Domínio do problema**
- Temos um conjunto de itens $I$ com $n$ itens, cada item $i$ contido em $I$ é caracterizado por:
    - seu peso $w_i$
    - seu valor $v_i$
- Mochila de capacidade máxima $K$

**2. Variáveis de decisão**
- $x_i$ denota se o item $i$ foi pego ou não
    - 1 se o item $i$ foi pego
    - 0 se o item $i$ não foi pego

**3. Condição de contorno**
- Os itens pegos não podem exceder a capacidade $K$ da mochila<br>
    $\sum_{i} x_i w_i <= K$
    
**4. Função Objetivo**
- Queremos maximizar o valor dos itens na mochila<br>
    $\sum_{i} x_i v_i$
    
## Soluções ótimas
Iremos explorar métodos que nos dão a solução ótima. Os métodos gulosos (pegar os itens mais valiosos/mais leves/mais densos até não caber mais) nos dão soluções que não quebram a condição de capacidade máxima, porém também não temos certeza de que o valor dos itens presentes na mochila é o maior possível. A seguir listaremos esses métodos com uma breve descrição deles.

## 1. Leitura dos dados
Diferentes configurações da mochila estão presentes na pasta data. Para cada arquivo, temos:
- Primeira linha contém a quantidade de itens e a capacidade da mochila, respectivamente
- A seguir, cada linha representa um item. Temos seu valor e peso, respectivamente

Vamos escolher o arquivo *ks_30_0* para exemplificar as soluções. A seguir temos uma função que faz a leitura desse arquivo.

In [1]:
from collections import namedtuple

def read_items(file_location: str) -> tuple:
    """Função usada para ler os arquivos que contém informações sobre a mochila.

    Args:
        file_location: string no formato "data/nome_arquivo"

    Returns:
        tupla contendo a capacidade da mochila, o número de itens e uma lista de namedtuple contendo os itens    
    """
    Item = namedtuple('Item', ['index', 'value', 'weight'])  # vamos armazenar os itens em uma lista de namedtuple
    
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
            
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []

    for i in range(1, item_count + 1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i - 1, int(parts[0]), int(parts[1])))
        
    return capacity, item_count, items

In [2]:
capacity, n_items, items = read_items('../data/ks_19_0')
capacity, n_items, items

(31181,
 19,
 [Item(index=0, value=1945, weight=4990),
  Item(index=1, value=321, weight=1142),
  Item(index=2, value=2945, weight=7390),
  Item(index=3, value=4136, weight=10372),
  Item(index=4, value=1107, weight=3114),
  Item(index=5, value=1022, weight=2744),
  Item(index=6, value=1101, weight=3102),
  Item(index=7, value=2890, weight=7280),
  Item(index=8, value=962, weight=2624),
  Item(index=9, value=1060, weight=3020),
  Item(index=10, value=805, weight=2310),
  Item(index=11, value=689, weight=2078),
  Item(index=12, value=1513, weight=3926),
  Item(index=13, value=3878, weight=9656),
  Item(index=14, value=13504, weight=32708),
  Item(index=15, value=1865, weight=4830),
  Item(index=16, value=667, weight=2034),
  Item(index=17, value=1833, weight=4766),
  Item(index=18, value=16553, weight=40006)])

## 1. Programação dinâmica
A ideia desse método é encontrar a solução ótima considerando um item de cada vez, para mochilas de capacidade $[0, ..., K]$. Vamos chamar de $O(k, j)$ a solução ótima da submochila de capacidade $k$ que considera apenas os itens $[0, ..., j]$ (o primeiro item é indexado em 0). Nosso interesse está em encontrar $O(K, n-1)$, que considera todos os itens e possui a capacidade da nossa mochila.<br>
- Assuma que saibamos resolver $O(k,j-1)$, para todo $k$ em $[0, ..., K]$
- Estamos interessados em encontrar $O(k, j)$, ou seja, estamos considerando apenas mais um item, $j$
- Se $w_j \le k$, podemos pegar ou não o item $j$
    - Se não pegarmos $j$, nossa melhor solução será $O(k, j-1)$ (solução ótima não considerando o item $j$)
    - Se pegarmos $j$, nossa melhor solução será $v_j + O(k-w_j, j-1)$ (valor do item $j$ acrescentado da solução ótima da mochila de capacidade $k-w_j$ sem considerar o item $j$
- Assim, temos:
    - $O(k, j) = max(O(k, j-1), v_j + O(k-w_j, j-1))$, se $w_j \le k$
    - $O(k-w_j, j-1)$ do contrário
    
Podemos então resolver o problema da mochila usando uma simples recursão:


In [3]:
def recursion_solution(k: int, j: int, items: list) -> float:
    """Função recursiva para encontrar a solução ótima para o problema da mochila.
    
    Args:
        k: tamanho da submochila
        j: itens considerados no subproblema (apenas os j primeiros)
        items: lista de namedtuple com todos os itens do problema
        
    Retorna:
        Solução ótima para a mochila de capacidade k, considerando apenas os j primeiros itens    
    """
    if j == -1:  # com j == -1 não estamos considerando nenhum item (começamos com index 0)
        return 0
    elif items[j].weight <= k:
        return max(
            recursion_solution(k, j - 1, items),
            items[j].value + recursion_solution(k - items[j].weight, j - 1, items)
        )
    else:
        return recursion_solution(k, j - 1, items)

In [4]:
solution = recursion_solution(capacity, n_items - 1, items)
print(f'Solução ótima: {solution}')

Solução ótima: 12248


Essa solução, no entanto, não é muito eficiente. Pense nela como uma função recursiva de Fibonacci, onde $fib(n) = fib(n-1) + fib(n-2)$, com as devidas condições iniciais. Veja que $fib(n-1) = fib(n-2) + fib(n-3)$. Porém $fib(n-2)$ já foi calculado anteriormente. Ou seja, estamos fazendo cálculos redundantes. Para uma mochila de grande capacidade e vários itens a solução leva um tempo exponencial, tornando-se inviável. Além disso, essa solução não nos dá quais itens foram pegos. Para verificar o problema da complexidade, rode o algoritmo com o arquivo *data/ks_40_0*.<br><br>
Uma forma de contornar essa inviabilidade é o uso de programação dinâmica. Nela, fazemos um trade off entre complexidade de tempo e memória, armazenando soluções já computadas (diminuição no tempo com um maior gasto de memória). Primeiramente, criamos uma tabela de tamanho $(n+1) x (K+1)$ (número de itens + 1 x capacidade da mochila + 1). A cada linha iremos considerar um item a mais no nosso conjunto:
- Na linha zero não consideramos nenhum item
- Na linha um consideramos apenas o primeiro item (index 0)
- Na linha dois consideramos os primeiro e segundo itens (index 0 e 1) etc
- Na última linha estaremos considerando todos os itens

Para as colunas temos a capacidade de submochilas da mochila principal:
- Na primeira coluna temos uma submochila de capacidade 0
- Na segunda coluna temos uma submochila de capacidade 1
- Na última coluna nossa mochila de capacidade $K$

A ideia é que cada posição dessa tabela armazene a solução ótima para aqueles itens/capacidade. Estamos então interessados na última linha e coluna da nossa tabela, que representa a capacidade máxima e considera todos os itens. Vamos preenchendo essa tabela da esquerda para a direita, de cima para baixo (para cada item adicionado resolvemos todas as submochilas).

In [5]:
def dynamic_programing_solver(items: list, capacity: int) -> tuple:
    """Resolve o problema da mochila através de programação dinâmica.
    
    Args:
        items: lista de namedtuple contendo todos os itens
        capacity: capacidade máxima da mochila
    
    Returns:
        S: tabela com todas as subsoluções
        taken: itens presentes na solução ótima
    """
    n_items = len(items)
    S = []
    for i in range(len(items) + 1):  # iniciando a tabela com zeros
        S += [[0]*(capacity + 1)]
        
    for i in range(n_items):  #  para cada item
        for k in range(capacity + 1):  # para todas as capacidades
            if items[i].weight <= k:  # se o item couber, podemos pegá-lo ou não
                S[i + 1][k] = max(
                    S[i + 1 - 1][k],
                    S[i + 1 - 1][k - items[i].weight] + items[i].value
                )  # lógica descrita acima. O item i é considerado na linha i + 1 (primeira linha considera nenhum item)
            else:  # se não couber, a solução é a mesma da que não considera esse item
                S[i + 1][k] = S[i + 1 - 1][k]
    
    # items presentes na solução ótima
    k = capacity
    taken = []
    for i in range(n_items - 1, -1, -1):  # começamos no último item/coluna
        if S[i + 1][k] != S[i + 1 - 1][k]:  # se o valor ótimo for diferente do imediatamente acima, o item foi pego
            taken.append(items[i].index)
            k = k - items[i].weight  # descontamos o peso do item
    
    return S, taken

In [6]:
solution_table, items_taken = dynamic_programing_solver(items, capacity)
print(f'Solução ótima: {solution_table[-1][-1]}, itens pegos: {items_taken}')

Solução ótima: 12248, itens pegos: [13, 12, 7, 5, 2]


O custo dessa solução é o custo da montagem dessa tabela. Ou seja, temos um custo de $O(nK)$ na memória e no tempo.

## 2. Branch and bound
**Branching** significar explorar as combinações de itens, pegando eles ou não. A figura abaixo mostra um exemplo de branching exaustivo:

![img](../images/exhaustive_search.png)

Vamos pensar que cada item avaliado é um nó, e devemos decidir se pegamos ele ou não. A única maneira de saber é explorando os dois nós filhos desse nó: o da "esquerda" que considera que o item foi pego e o da "direita" que considera que o item não foi pego.
Essa exploração é, no entanto, cara (complexidade exponencial com a quantidade de itens).<br><br>
**Bounding** é uma estimativa que nos permite parar ou continuar a exploração descrita acima, podando uma parte do espaço de busca. Vamos usar o *relaxation* em nosso Bounding. Nesse método consideramos que podemos partes fracionárias dos itens. Com isso, temos uma "solução otimista", já que estamos pegando os itens mais "densos" até não caber mais (precisamos primeiramente ordenar os itens por densidade de valor, do mais denso ao menos). Usamos isso para um critério de parada na exploração da árvore.<br>

Podemos então melhorar nossa exploração exaustiva adicionando alguns critérios de parada, como:
- se o nó explorado possuir capacidade negativa (ou seja, estouramos a capacidade da mochila)
- se a solução otimista para o nó for pior que a melhor solução encontrada até o momento

Para poder usar esses critérios, devemos ter as seguintes informações para cada nó explorado:
- Qual item o nó está avaliando
- Quais itens pegamos até chegar a esse nó (para atualizarmos nossa melhor solução com eles, quando necessário)
- Qual o valor agregado nesse nó (soma dos valores dos itens pegos, pelo mesmo motivo acima)
- Qual nossa solução otimista para esse nó (para usarmos como parada na exploração)
- Qual a capacidade vaga que temos no nó (capacidade total menos os pesos dos itens pegos, usada como critério de parada)

Além disso, precisamos trackear a melhor solução até o momento:
- Valor da melhor solução
- Itens pegos na melhor solução

Ao chegar em um nó, devemos decidir explorá-lo ou não: ou seja, examinar seus filhos ou não. O algoritmo de exploração baseia-se em:
- Se o nó em questão possui capacidade negativa, paramos a busca
- Se a solução otimista for menor que a melhor solução até o momento, não o exploramos
- Se chegarmos ao último item, paramos a busca
- Sempre que um nó possuir valor maior que o da melhor solução até o momento devemos atualizar a melhor solução

Tendo dito tudo isso, devemos definir uma estratégia de busca. Qual nó exploramos antes, o da esquerda ou da direita?

## 2.1 Depth-First Branch and Bound
Nessa estratégia exploramos sempre o nó a esquerda, ou seja, sempre pegamos o item.

In [7]:
class NodeExplorer():
    """Cria objetos que representem os nós.
    """
    def __init__(self, i: int, items_taken: list, value: float, capacity: float, best_estimation: float):
        """Inicializador do nó.
        
        Args:
            i: índice do item a ser explorado no nó.
            items_taken: itens pegos até o momento, no formato 0 ou 1.
            value: valor agregado no nó (soma dos itens pegos).
            capacity: capacidade disponível para o nó (capacidade total menos o peso dos itens pegos).
            best_estimation: solução otimista para aquele nó.
        """
        self.i = i
        self.items_taken = items_taken
        self.value = value
        self.best_estimation = best_estimation
        self.capacity = capacity

class DepthFirstSearch():
    """Orquestrador da estratégia de busca.
    """
    
    def __init__(self, items: list, capacity: float):
        """Inicializador da busca.
        
        Args:
            items: conjunto de itens a ser explorado.
            capacity: capacidade máxima da mochila.
        """
        self.items = items
        self.capacity = capacity
        self.best = 0  # iniciamos nossa melhor solução com valor nulo
        self.items_taken = []  # e sem pegar nenhum item
        
        self.start_search()  # começa a busca
        self.collect_items()  # verifica quais itens foram pegos


    def sorting(self):
        """Ordena os itens por densidade de valor, da maior para a menor.
        """
        self.items = sorted(self.items, key=lambda x: x.weight/x.value)

    def best_possible_solution(self, items: list, k: float) -> float:
        """Retorna a solução otimista para um nó (usando relaxation de itens fracionários).
        
        Args:
            items: lista contendo o subconjunto de itens que serão analizados no nó (e seus nós filhos) em questão.
            k: capacidade disponível no nó.
            
        Returns:
            solução ótima para a configuração acima.
        """
        best_value = 0
        i = 0
        while k > 0 and i < len(items):  # enquanto ouver espaço na mochila, pegamos itens
            best_value += items[i].value
            k -= items[i].weight
            i += 1
        
        if k < 0:  # se o último item pego estourar a capacidade, pegamos apenas uma fração dele
            i -= 1
            best_value -= items[i].value
            k += items[i].weight
            best_value += k/items[i].weight*items[i].value
  
        return best_value

    def searchNode(self, node: object):
        """Faz a análise e exploração de um nó.
        
        Args:
            node: objeto que representa um nó.
        """
        if node.capacity < 0 or node.best_estimation < self.best:  # critérios de parada
            return None
        elif node.value > self.best:  # caso encontramos um valor maior que o melhor valor até o momento
            self.best = node.value
            self.items_taken = node.items_taken
        if node.i == len(self.items):  # no último item não exploramos seus filhos
            return None
        
        # nó a esquerda, no qual pegamos o item em questão
        node1 = NodeExplorer(
            node.i + 1,  # próximo item
            node.items_taken + [1],  # item pego
            node.value + self.items[node.i].value,  # adiciona o valor do item
            node.capacity - self.items[node.i].weight,  # remove da capacidade o peso do item
            node.best_estimation  # nossa melhor estimativa não muda, já que pegamos o item mais denso
        )
        
        # nó a direita, no qual não pegamos o item em questão
        node0 = NodeExplorer(
            node.i + 1,
            node.items_taken + [0],  # item não pego
            node.value,  # não adicionamos o valor do item
            node.capacity,  # capacidade disponível não muda, já que não pegamos o item
            node.value + self.best_possible_solution(self.items[node.i + 1:], node.capacity)  # não pegar o item mais denso nos  leva a atualizar a solução ótima, que será o valor do nó somado a solução ótima do filho
        )
        
        # depth-first: buscamos primeiramente sempre à esquerda
        self.searchNode(node1)
        self.searchNode(node0)
        
    def start_search(self):
        """Inicia a busca no primeiro nó.
        """
        self.sorting()  # ordenar para termos os itens mais valiosos no início (e podermos usar o relaxation)
        starterNode = NodeExplorer(
            0,  # começamos no primeiro item da lista
            [],  # não pegamos nenhum item
            0,  # nosso valor inicial é 0 (não pegamos nada ainda)
            self.capacity,  # começamos com a capacidade máxima da mochila
            self.best_possible_solution(self.items, self.capacity)  # solução otimista considera todos os items
        )
        self.searchNode(starterNode)

    def collect_items(self):
        """Verifica quais itens foram pegos.
        """
        items_taken = []
        for i in range(len(self.items_taken)):
            if self.items_taken[i] == 1:
                items_taken.append(self.items[i].index)
        self.items_taken = items_taken

In [8]:
depth_first = DepthFirstSearch(items, capacity)
print(f'Solução ótima: {depth_first.best}, itens pegos: {depth_first.items_taken}')

Solução ótima: 12248, itens pegos: [13, 2, 7, 12, 5]


## 2.2 Best-First Branch and Bound
Nessa estratégia exploramos sempre o nó de maior potencial, o que tem a melhor solução otimista. Para isso, devemos sempre ter mapeado o nó com a melhor solução otimista: podemos utilizar uma fila de prioridade. Ao chegar em um nó, adicionamos seus filhos a fila de prioridade. Na hora de decidir o próximo nó a ser explorado simplesmente damos um pop nessa fila de prioridade.

In [9]:
class MaxPQ():
    """Fila de prioridade de máximo. Utilizaremos um vetor para representar a fila.
        
        Nesse vetor, o índice i será pai dos índices 2*i e 2*i + 1. Um filho nunca poderá ser maior que seu pai.
    """
    def __init__(self):
        """Inicializador da fila.
        """
        self.pq = [0]  # para falicitar as operações, a posição inicial do vetor não será usuada
        self.length = 0
        
    def isEmpty(self):
        """Verifica se a fila está vazia.
        """
        return self.length == 0
    
    def swim(self, k: int):
        """Função suporte que atualiza a posição de um elemento na fila, quando ele possui pais menores que ele.
        
        Args:
            k: índice do elemento.
        """
        # como estamos falando de nós (definido no depth-first) devemos comparar o atributo best_estimation
        while k > 1 and self.pq[k//2].best_estimation < self.pq[k].best_estimation:  # enquanto o filho for maior que o pai
            self.pq[k//2], self.pq[k] = self.pq[k], self.pq[k//2]  # trocamos as posições
            k //= 2
    
    def insert(self, element: object):
        """Insere um novo nó na fila.
        
        Args:
            element: nó (objeto NodeExplorer).
        """
        self.length += 1
        self.pq.append(element)  # adicionamos sempre na última posição da lista
        self.swim(self.length)  # achamos a posição ideal para o nó adicionado
        
    def sink(self, k: int):
        """Função suporte que atualiza a posição de um elemento na fila, quando ele possui filhos maiores que ele.
        
        Args:
            k: índice do elemento.
        """
        while 2*k <= self.length:  # enquanto possuir filhos
            j = 2*k  # veja o filho da esquerda
            if j < self.length and self.pq[j].best_estimation < self.pq[j + 1].best_estimation:  # se possuir dois filhos
                j += 1  # aponte para o filho da direita caso ele seja maior
            if self.pq[k].best_estimation >= self.pq[j].best_estimation:  # se o pai for maior que o filho pare
                break
            self.pq[k], self.pq[j] = self.pq[j], self.pq[k]  # troque pai com filho
            k = j  # atualize o índice do elemento que estamos trabalhando
    
    def getMax(self) -> object:
        """Remove e retorna o objeto com o maior best_estimation.
        
        Returns:
            Nó com o maior best_estimation.
        """
        self.pq[1], self.pq[-1] = self.pq[-1], self.pq[1]  # troca o maior elemento pelo último
        obj = self.pq.pop()  # retira o último elemento (que agora é o maior)
        self.length -= 1
        self.sink(1)  # atualiza a posição do agora primeiro elemento
        
        return obj  # retorna o elemento

Com nossa fila de prioridade definida, vamos definir nossa classe responsável pela best-first search. Essa classe é muito parecida com a classe anterior (DepthFirstSearch). A única mudança é na chamada de exploração de novos nós (método *searchNode*) e no atributo *priority_queue*, que armazenará a fila de prioridade.

In [10]:
class BestFirstSearch():
    """Orquestrador da estratégia de busca.    
    """
    
    def __init__(self, items: list, capacity: float):
        """Inicializador.
        
        Args:
            items: conjunto de itens a ser explorado.
            capacity: capacidade máxima da mochila.
        """        
        self.items = items
        self.capacity = capacity
        self.best = 0  # iniciamos nossa melhor solução com valor nulo
        self.items_taken = []  # e sem pegar nenhum item
        self.priority_queue = MaxPQ()
        
        self.start_search()  # começa a busca
        self.collect_items()  # verifica quais itens foram pegos

    def sorting(self):
        """Ordena os itens por densidade de valor, da maior para a menor.
        """
        self.items = sorted(self.items, key=lambda x: x.weight/x.value)

    def best_possible_solution(self, items: list, k: float) -> float:
        """Retorna a solução otimista para um nó (usando relaxation de itens fracionários).
        
        Args:
            items: lista contendo o subconjunto de itens que serão analizados no nó (e seus nós filhos) em questão.
            k: capacidade disponível no nó.
            
        Returns:
            solução ótima para a configuração acima.
        """
        best_value = 0
        i = 0
        while k > 0 and i < len(items):  # enquanto ouver espaço na mochila, pegamos itens
            best_value += items[i].value
            k -= items[i].weight
            i += 1
        
        if k < 0:  # se o último item pego estourar a capacidade, pegamos apenas uma fração dele
            i -= 1
            best_value -= items[i].value
            k += items[i].weight
            best_value += k/items[i].weight*items[i].value
  
        return best_value

    def searchNode(self, node: object):
        """Faz a análise e exploração de um nó.
        
        Args:
            node: objeto que representa um nó.
        """
        if node.capacity < 0 or node.best_estimation < self.best:  # critérios de parada
            return None
        elif node.value > self.best:  # caso encontramos um valor maior que o melhor valor até o momento
            self.best = node.value
            self.items_taken = node.items_taken
        if node.i == len(self.items):  # no último item não exploramos seus filhos
            return None
        
        # nó a esquerda, no qual pegamos o item em questão
        node1 = NodeExplorer(
            node.i + 1,  # próximo item
            node.items_taken + [1],  # item pego
            node.value + self.items[node.i].value,  # adiciona o valor do item
            node.capacity - self.items[node.i].weight,  # remove da capacidade o peso do item
            node.best_estimation  # nossa melhor estimativa não muda, já que pegamos o item mais denso
        )
        
        # nó a direita, no qual não pegamos o item em questão
        node0 = NodeExplorer(
            node.i + 1,
            node.items_taken + [0],  # item não pego
            node.value,  # não adicionamos o valor do item
            node.capacity,  # capacidade disponível não muda, já que não pegamos o item
            node.value + self.best_possible_solution(self.items[node.i + 1:], node.capacity)  # não pegar o item mais denso nos  leva a atualizar a solução ótima, que será o valor do nó somado a solução ótima do filho
        )
        
        # adicionando os dois nós na fila de prioridade
        self.priority_queue.insert(node1)
        self.priority_queue.insert(node0)
        
        # enquanto a fila não estiver vazia e a melhor solução otimista for maior que a melhor solução
        while not(self.priority_queue.isEmpty()) and self.priority_queue.pq[1].best_estimation > self.best:
            self.searchNode(self.priority_queue.getMax())  # o próximo nó a ser explorado é o pop da fila     
        
    def start_search(self):
        """Inicia a busca no primeiro nó.
        """
        self.sorting()  # ordenar para termos os itens mais valiosos no início (e podermos usar o relaxation)
        starterNode = NodeExplorer(
            0,  # começamos no primeiro item da lista
            [],  # não pegamos nenhum item
            0,  # nosso valor inicial é 0 (não pegamos nada ainda)
            self.capacity,  # começamos com a capacidade máxima da mochila
            self.best_possible_solution(self.items, self.capacity)  # solução otimista considera todos os items
        )
        self.searchNode(starterNode)

    def collect_items(self):
        """Verifica quais itens foram pegos.
        """
        items_taken = []
        for i in range(len(self.items_taken)):
            if self.items_taken[i] == 1:
                items_taken.append(self.items[i].index)
        self.items_taken = items_taken

In [11]:
best_first = BestFirstSearch(items, capacity)
print(f'Solução ótima: {best_first.best}, itens pegos: {best_first.items_taken}')

Solução ótima: 12248, itens pegos: [13, 2, 7, 12, 5]


O ponto negativo dessa solução é a memória que ela ocupa. Exagerando, pense que todos os últimos nós tenham a mesma solução ótima. Iríamos ter de armazenar todos eles, ocupando uma memória de ordem exponencial. Experimente rodar essa busca com *data/ks_40_0*, provavelmente irá estourar a memória.

## 3. Referências
[Curso de Otimização discreta do Coursera](https://www.coursera.org/learn/discrete-optimization)